<a href="https://colab.research.google.com/github/gianmarco-holm/PY02_MachineLearning_LinearRegression_Titanic/blob/main/PY02_MachineLearning_LinearRegression_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Regresión Lineal de sobrevivientes de Titanic

Este proyecto utiliza técnicas de machine learning, específicamente un modelo de regresión lineal para poder predecir si una persona puede sobrevivir o no al titanic

## Carga de Datos

In [ ]:
# Este comando solo se ejecuta una vez para actualizar pip y pueda usarse la librería stats, luego se vuelve a comentar
#!pip install --upgrade pip setuptools==57.5.0

In [ ]:
# Comando que nos permite instalar la librería que tiene a stats
!pip install regressors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importar librerías

# Librerías de transformación
import pandas as pd
import numpy as np

# Librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Librerías de ML
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from regressors import stats
from sklearn.model_selection import cross_val_score
import pickle
import re

In [ ]:
# Asignar estilo a seaborn
sns.set(style = 'darkgrid', context = 'notebook')

In [ ]:
# Extraer datos
df = pd.read_csv('./drive/MyDrive/machine_learning/data/titanic.csv')
df.head()

,IdPasajero,survival,pclass,nombre,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",masculino,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",femenino,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",femenino,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",femenino,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",masculino,35.0,0,0,373450,8.0500,NaN,S


**Definición funcional:**
1. IdPasajero: Un identificador único para cada pasajero.
2. survival: Indica si el pasajero sobrevivió (1) o no (0).
3. pclass: Clase del pasajero en el barco (1 = Primera clase, 2 = Segunda clase, 3 = Tercera clase).
4. nombre: Nombre completo del pasajero.
5. sex: Género del pasajero (masculino o femenino).
6. age: Edad del pasajero en años. Puede contener valores nulos si la edad no se conoce.
7. sibsp: Número de hermanos o cónyuges del pasajero a bordo del Titanic.
8. parch: Número de padres o hijos del pasajero a bordo del Titanic.
9 ticket: Número del billete del pasajero.
10. fare: Tarifa pagada por el billete en libras esterlinas.
11. cabin: Número de cabina del pasajero. Puede contener valores nulos si la cabina no se conoce.
12. embarked: Puerto de embarque del pasajero (C = Cherburgo, Q = Queenstown, S = Southampton).

## Análisis exploratorio

In [ ]:
# Información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   IdPasajero  891 non-null    int64  
 1   survival    891 non-null    int64  
 2   pclass      891 non-null    int64  
 3   nombre      891 non-null    object 
 4   sex         891 non-null    object 
 5   age         714 non-null    float64
 6   sibsp       891 non-null    int64  
 7   parch       891 non-null    int64  
 8   ticket      891 non-null    object 
 9   fare        891 non-null    float64
 10  cabin       204 non-null    object 
 11  embarked    889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# Estadísticas descriptiva
df.describe()

,IdPasajero,survival,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# Comprobar valores faltantes
df.isnull().sum()

IdPasajero      0
survival        0
pclass          0
nombre          0
sex             0
age           177
sibsp           0
parch           0
ticket          0
fare            0
cabin         687
embarked        2
dtype: int64

In [ ]:
# Manejo de valores faltantes
# Rellenar la edad con la media de edades
df['age'].fillna(df['age'].mean(), inplace=True)

# Rellenar los valores nulos de cabin con unknown
df['cabin'].fillna('Unknown', inplace=True)
# Es importante agregar una nueva categoría en este caso que represente la ausencia de datos en lugar en lugar de eliminarlos
# ya que esta ausencia puede estar relacionada con alguna variable del dataset, por ejemplo que su clase sea bien baja

# Eliminar los valores nulos de embarked
df.dropna(subset=['embarked'], inplace=True)

In [ ]:
# Comprar los valores faltantes
df.isnull().sum()

IdPasajero    0
survival      0
pclass        0
nombre        0
sex           0
age           0
sibsp         0
parch         0
ticket        0
fare          0
cabin         0
embarked      0
dtype: int64

In [ ]:
# Extraer el título del nombre del pasajero
df['title'] = df['nombre'].apply(lambda x: re.search(', (.*?)\.', x).group(1))
df['family'] = df.sibsp*df.parch

In [ ]:
# Crear variables dummy para las características categóricas
df = pd.get_dummies(df, columns=['sex', 'embarked', 'title'], drop_first=True)

In [ ]:
df

,IdPasajero,survival,pclass,nombre,age,sibsp,parch,ticket,fare,cabin,...,title_Master,title_Miss,title_Mlle,title_Mme,title_Mr,title_Mrs,title_Ms,title_Rev,title_Sir,title_the Countess
0,1,0,3,"Braund, Mr. Owen Harris",22.000000,1,0,A/5 21171,7.2500,Unknown,...,False,False,False,False,True,False,False,False,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.000000,1,0,PC 17599,71.2833,C85,...,False,False,False,False,False,True,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",26.000000,0,0,STON/O2. 3101282,7.9250,Unknown,...,False,True,False,False,False,False,False,False,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.000000,1,0,113803,53.1000,C123,...,False,False,False,False,False,True,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",35.000000,0,0,373450,8.0500,Unknown,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.000000,0,0,211536,13.0000,Unknown,...,False,False,False,False,False,False,False,True,False,False
887,888,1,1,"Graham, Miss. Margaret Edith",19.000000,0,0,112053,30.0000,B42,...,False,True,False,False,False,False,False,False,False,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",29.699118,1,2,W./C. 6607,23.4500,Unknown,...,False,True,False,False,False,False,False,False,False,False
889,890,1,1,"Behr, Mr. Karl Howell",26.000000,0,0,111369,30.0000,C148,...,False,False,False,False,True,False,False,False,False,False


In [ ]:
# Seleccionar las variables independientes
X_cols = ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_masculino', 'title_Mr', 'title_Mrs', 'title_Miss','family']
y_cols = 'survival'

In [ ]:
cols = X_cols + [y_cols]

In [ ]:
# Crear la correlación de variables
matriz_correlacion = df[cols].corr()

In [ ]:
# Analisis visual para encontrar relación entre las variables
# Crear el heatmap usando imshow
fig_heatmap = px.imshow(
    matriz_correlacion,
    text_auto=True,
    aspect="auto",
    color_continuous_scale='Viridis',
    title='Heatmap de la Matriz de Correlación'
)

# Mostrar el gráfico
fig_heatmap.show()

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df[y_cols], test_size=0.2, random_state=42)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Ajustamos las estadísticas y transformamos
X_test_scaled = scaler.transform(X_test) # Con las estadísticas ya ajustado, aca solo trasnformamos

# Crear el modelo
model = LinearRegression()
model.fit(X_train_scaled, y_train) # Entrenamos el modelo

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Crear un DataFrame con los resultados
results_df = X_test.copy()
results_df['Valores Reales'] = y_test.values
results_df['Valores Predichos'] = y_pred
results_df

,pclass,age,sibsp,parch,fare,sex_masculino,title_Mr,title_Mrs,title_Miss,family,Valores Reales,Valores Predichos
281,3,28.0,0,0,7.8542,True,True,False,False,0,0,0.110252
435,1,14.0,1,2,120.0000,False,False,False,True,2,1,0.913534
39,3,14.0,1,0,11.2417,False,False,False,True,0,1,0.633909
418,2,30.0,0,0,13.0000,True,True,False,False,0,0,0.252274
585,1,18.0,0,2,79.6500,False,False,False,True,0,1,0.933296
...,...,...,...,...,...,...,...,...,...,...,...,...
433,3,17.0,0,0,7.1250,True,True,False,False,0,0,0.170074
807,3,18.0,0,0,7.7750,False,False,False,True,0,0,0.676718
25,3,38.0,1,5,31.3875,False,False,True,False,5,1,0.434393
85,3,33.0,3,0,15.8500,False,False,True,False,0,1,0.543055


In [ ]:
# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Error cuadrático medio (MSE): {mse:.2f}')
print(f'Coeficiente de determinación (R^2): {r2:.2f}')

Error cuadrático medio (MSE): 0.15
Coeficiente de determinación (R^2): 0.38


In [ ]:
# Calcular estadísticas para evaluar el modelo a mayor detalle
print('========= Summary =========')
stats.summary(model, X_test_scaled, y_test, X_cols)

# Validación cruzada
cv_scores = cross_val_score(model,X_train_scaled, y_train, cv=10, scoring='r2')
print(f'Validación cruzada R²: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}')

========= Summary =========
Residuals:
    Min     1Q  Median     3Q    Max
-0.1897 0.1201  0.3877 0.7242 1.1243


Coefficients:
               Estimate  Std. Error  t value   p value
_intercept     0.381153    0.029792  12.7939  0.000000
pclass        -0.124899    0.035078  -3.5607  0.000475
age           -0.071877    0.034035  -2.1118  0.036106
sibsp         -0.078219    0.064747  -1.2081  0.228629
parch         -0.035901    0.034704  -1.0345  0.302317
fare           0.031835    0.031940   0.9967  0.320258
sex_masculino -0.116725    0.115105  -1.0141  0.311934
title_Mr      -0.181052    0.060320  -3.0015  0.003075
title_Mrs      0.013767    0.067037   0.2054  0.837521
title_Miss    -0.041166    0.079762  -0.5161  0.606423
family        -0.000889    0.089238  -0.0100  0.992061
---
R-squared:  0.38345,    Adjusted R-squared:  0.34653
F-statistic: 10.39 on 10 features
Validación cruzada R²: 0.41 ± 0.08


In [ ]:
# Gráfico de residuos
residuals = y_test - y_pred
fig_residuals = px.scatter(x=y_pred, y=residuals, title='Gráfico de Residuos')
fig_residuals.add_shape(type='line', x0=y_pred.min(), y0=0, x1=y_pred.max(), y1=0,
                        line=dict(color='Red', dash='dash'))
fig_residuals.update_layout(
    xaxis_title="Valores Predichos",
    yaxis_title="Residuos"
)
fig_residuals.show()

## Conclusiones

- Realizamos manejo de valores faltantes para mejorar el R2.
- Tambien creamos nuevas variables para mejorar los indicadores.
- Sin embargo, no pudimos mejorar el modelo de ML, esto significa que debemos de intentar con otros tipo de tecnicas para este tipo de caso.